## Location Data Extraction



In [ ]:
import pandas as pd
import numpy as np

datafile = '../data/fl.csv'

df = pd.read_csv(datafile)
df.head(20)

In [ ]:
df_florida = df[ df['Lossstate']=='florida' ]
df_florida.shape

In [ ]:
df_florida.head()

In [ ]:
df_florida.iloc[0]

In [ ]:
# df_florida.to_csv('florida_locations_raw.csv', index=False)

In [ ]:
df_florida['Lossstreet'] = df_florida['Lossstreet'].replace(r'\brd\b','road', regex=True) \
                                                    .replace(r'\bst\b','street', regex=True) \
                                                    .replace(r'\bave\b','avenue', regex=True).copy()
df_florida.head()

In [ ]:
df_florida['loss_details'] = df_florida['loss_details'].replace(r'\brd\b','road', regex=True) \
                                                    .replace(r'\bst\b','street', regex=True) \
                                                    .replace(r'\bave\b','avenue', regex=True).copy()
df_florida.head()

Replace rd, st, and ave wiith road, street, and avenue respectively

In [ ]:
# loss location level2

df_florida['loss location level2'] = df_florida['loss location level2'].replace(r'\brd\b','road', regex=True) \
                                                    .replace(r'\bst\b','street', regex=True) \
                                                    .replace(r'\bave\b','avenue', regex=True).copy()
df_florida.head()

Substitute numbers with words

In [ ]:
def num2words(num):
    under_20 = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Eleven','Twelve','Thirteen','Fourteen','Fifteen','Sixteen','Seventeen','Eighteen','Nineteen']
    tens = ['Twenty','Thirty','Forty','Fifty','Sixty','Seventy','Eighty','Ninety']
    above_100 = {100: 'Hundred',1000:'Thousand', 1000000:'Million', 1000000000:'Billion'}
 
    if num < 20:
        return under_20[num]

    if num < 100:
        return tens[(int)(num/10)-2] + ('' if num%10==0 else ' ' + under_20[num%10])
 
    # find the appropriate pivot - 'Million' in 3,603,550, or 'Thousand' in 603,550
    pivot = max([key for key in above_100.keys() if key <= num])
 
    return num2words((int)(num/pivot)) + ' ' + above_100[pivot] + ('' if num%pivot==0 else ' ' + num2words(num%pivot))


In [ ]:
def num2words_x2(num):
    under_20 = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Eleven','Twelve','Thirteen','Fourteen','Fifteen','Sixteen','Seventeen','Eighteen','Nineteen']
    
    num_s = ""
    for i in str(num):
        num_s = num_s + ' ' + under_20[ int(i)]

    print(num_s.strip())
    return num_s.strip()


In [ ]:
s = "state road 429"
def number_replace(match):
    num = match.group()
    return num2words(int(num)).lower()

re.sub(r'([0-9]+)', number_replace, s)


In [ ]:
lossstreet_n2w = []
for index, row in df_florida.iterrows():
    Lossstreet = row['Lossstreet']
    Lossstreet_w = re.sub(r'([0-9]+)', number_replace, Lossstreet)
    print( Lossstreet, Lossstreet_w)
    lossstreet_n2w.append(Lossstreet_w)
    
#     count += 1
#     if count == 10:
#        break

In [ ]:
df_florida['lossstreet_n2w']=lossstreet_n2w

In [ ]:
df_florida.to_csv('../data/florida_locations.csv', index=False)

In [ ]:
df = pd.read_csv('../data/florida_locations.csv')
df

In [ ]:
df['loss location level2'][0]

In [ ]:
df['loss_details'][0]

In [ ]:
import re

count = 0

print(df.columns)
#annotations = []
dir_out = './street_annotations/'

for index, row in df.iterrows():
    RecordingID = row['RecordingID']
    Losscity = row['Losscity']
    Lossstate = row['Lossstate']
    Mapaddress = row['Mapaddress']
    Lossstreet = row['Lossstreet']
    loss_details = row['loss_details']
    loss_location_level2 = row['loss location level2']
    lossstreet_n2w = row['lossstreet_n2w']
#     print(lossstreet_n2w, [Lossstreet, loss_location_level2])
    annotation_string = ""
    tcount=1
    end = -1
    lossstreet_n2w = Lossstreet
    start = loss_location_level2.find(lossstreet_n2w)
    print('---')
    print( loss_location_level2 )
    if start == -1:
        print( lossstreet_n2w )
        lossstreet_n2w = re.sub(r'([0-9]+)', '', lossstreet_n2w).strip()
        print( lossstreet_n2w )
        start = loss_location_level2.find(lossstreet_n2w)
    if start > -1:
        end = start + len(lossstreet_n2w)
        annotation_string = 'T'+str(tcount)+'\t'+'STREET'+' '+str(start)+' '+str(end)+'\t'+lossstreet_n2w
        tcount += 1
        print(annotation_string, ',', Lossstreet)
        #annotations.append((RecordingID, s, loss_location_level2))
    else:
        print('nada')
        
    fnametxt = str(RecordingID)+'.txt'
    fnameann = str(RecordingID)+'.ann'
    f = dir_out+fnametxt
    ftxt = open(f,"w+",encoding='utf-8')
    print(loss_location_level2, file=ftxt)
    ftxt.close()
    f = dir_out+fnameann
    fann = open(f,"w+",encoding='utf-8')
    print(annotation_string, file=fann)
    fann.close()
    

In [ ]:
s = '1771 ponkan road'
x = re.sub(r'([0-9]+)', '', s).strip()
s.find(x)

In [ ]:
#!mkdir street_annotations